[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openlayer-ai/openlayer-python/blob/main/examples/tracing/langchain/async_langchain_callback.ipynb)

# <a id="top">Openlayer Async LangChain Callback Handler</a>

This notebook demonstrates how to use Openlayer's **AsyncOpenlayerHandler** to monitor async LLMs, chains, tools, and agents built with LangChain.

The AsyncOpenlayerHandler provides:
- Full async/await support for non-blocking operations
- Proper trace management in async environments
- Support for concurrent LangChain operations
- Comprehensive monitoring of async chains, tools, and agents


## 1. Installation

Install the required packages:


In [ ]:
%pip install openlayer langchain langchain_openai langchain_community


## 2. Environment Setup

Configure your API keys and Openlayer settings:


In [ ]:
import os
import asyncio
from typing import List, Dict, Any

# OpenAI API key
os.environ["OPENAI_API_KEY"] = ""

# Openlayer configuration
os.environ["OPENLAYER_API_KEY"] = ""
os.environ["OPENLAYER_INFERENCE_PIPELINE_ID"] = ""


## 3. Instantiate the AsyncOpenlayerHandler

Create the async callback handler:


In [ ]:
from openlayer.lib.integrations import langchain_callback

# Create the async callback handler
async_openlayer_handler = langchain_callback.AsyncOpenlayerHandler(
    # Optional: Add custom metadata that will be attached to all traces
    user_id="demo_user",
    environment="development",
    session_id="async_langchain_demo"
)

print("AsyncOpenlayerHandler created successfully!")


## 4. Basic Async Chat Example

Let's start with a simple async chat completion:


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

async def basic_async_chat():
    """Demonstrate basic async chat with tracing."""
    
    # Create async chat model with callback
    chat = ChatOpenAI(
        model="gpt-3.5-turbo",
        max_tokens=100,
        temperature=0.7,
        callbacks=[async_openlayer_handler]
    )
    
    # Single async invocation
    print("🤖 Single async chat completion...")
    messages = [
        SystemMessage(content="You are a helpful AI assistant."),
        HumanMessage(content="What are the benefits of async programming in Python?")
    ]
    
    response = await chat.ainvoke(messages)
    print(f"Response: {response.content}")
    
    return response

# Run the basic example
response = await basic_async_chat()
print("\n✅ Basic async chat completed and traced!")


## 5. Concurrent Async Operations

Demonstrate the power of async with concurrent operations:


In [ ]:
async def concurrent_chat_operations():
    """Demonstrate concurrent async chat operations with individual tracing."""
    
    chat = ChatOpenAI(
        model="gpt-3.5-turbo",
        max_tokens=75,
        temperature=0.5,
        callbacks=[async_openlayer_handler]
    )
    
    # Define multiple questions to ask concurrently
    questions = [
        "What is machine learning?",
        "Explain quantum computing in simple terms.",
        "What are the benefits of renewable energy?",
        "How does blockchain technology work?"
    ]
    
    print(f"🚀 Starting {len(questions)} concurrent chat operations...")
    
    # Create concurrent tasks
    tasks = []
    for i, question in enumerate(questions):
        messages = [
            SystemMessage(content=f"You are expert #{i+1}. Give a concise answer."),
            HumanMessage(content=question)
        ]
        task = chat.ainvoke(messages)
        tasks.append((question, task))
    
    # Execute all tasks concurrently
    import time
    start_time = time.time()
    
    results = await asyncio.gather(*[task for _, task in tasks])
    
    end_time = time.time()
    
    # Display results
    print(f"\n⚡ Completed {len(questions)} operations in {end_time - start_time:.2f} seconds")
    for i, (question, result) in enumerate(zip([q for q, _ in tasks], results)):
        print(f"\n❓ Q{i+1}: {question}")
        print(f"💡 A{i+1}: {result.content[:100]}...")
    
    return results

# Run concurrent operations
concurrent_results = await concurrent_chat_operations()
print("\n✅ Concurrent operations completed and all traced separately!")


## 6. Async Streaming Example

Demonstrate async streaming with token-by-token generation:


In [ ]:
async def async_streaming_example():
    """Demonstrate async streaming with tracing."""
    
    # Create streaming chat model
    streaming_chat = ChatOpenAI(
        model="gpt-3.5-turbo",
        max_tokens=200,
        temperature=0.7,
        streaming=True,
        callbacks=[async_openlayer_handler]
    )
    
    print("🌊 Starting async streaming...")
    
    messages = [
        SystemMessage(content="You are a creative storyteller."),
        HumanMessage(content="Tell me a short story about a robot learning to paint.")
    ]
    
    # Stream the response
    full_response = ""
    async for chunk in streaming_chat.astream(messages):
        if chunk.content:
            print(chunk.content, end="", flush=True)
            full_response += chunk.content
    
    print("\n")
    return full_response

# Run streaming example
streaming_result = await async_streaming_example()
print("\n✅ Async streaming completed and traced!")


## 7. Async Chain Example

Create and run an async chain with proper tracing:


In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

async def async_chain_example():
    """Demonstrate async LLM chain with tracing."""
    
    # Create LLM with callback
    llm = OpenAI(
        model="gpt-3.5-turbo-instruct",
        max_tokens=150,
        temperature=0.8,
        callbacks=[async_openlayer_handler]
    )
    
    # Create a prompt template
    prompt = PromptTemplate(
        input_variables=["topic", "audience"],
        template="""
        Write a brief explanation about {topic} for {audience}.
        Make it engaging and easy to understand.
        
        Topic: {topic}
        Audience: {audience}
        
        Explanation:
        """
    )
    
    # Create the chain
    chain = LLMChain(
        llm=llm,
        prompt=prompt,
        callbacks=[async_openlayer_handler]
    )
    
    print("🔗 Running async chain...")
    
    # Run the chain asynchronously
    result = await chain.arun(
        topic="artificial intelligence",
        audience="high school students"
    )
    
    print(f"Chain result: {result}")
    return result

# Run the chain example
chain_result = await async_chain_example()
print("\n✅ Async chain completed and traced!")
